# Calibration backend

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

def calibrate(axis, voltage):
    """
    This function calibrates a given axis based on a given voltage. It reads data from a CSV file,
    fits a linear regression model to the data, and calculates the calibrated value and its uncertainty.

    Parameters:
    axis (str): The axis to be calibrated. Must be 'x', 'y', or 'z'.
    voltage (float): The voltage value used for calibration.

    Returns:
    None: This function does not return a value. It prints the calibrated value with its uncertainty.

    Raises:
    ValueError: If the axis is not a string or is not 'x', 'y', or 'z'.
    """
    # Check if axis is a string
    if not isinstance(axis, str):
        raise ValueError("Axis must be a string (x, y, or z)")
    
    # Read the data from the CSV file and skip every other row
    df = pd.read_csv('stage_data_final.csv')

    # Define the function for a linear regression line
    def linear_regression(v, a, b):
        return a * v + b
    
    params, covariance = curve_fit(linear_regression, df['volts (V)'], df[f'actual_distance_{axis}(um)'])

    # Errors in slope and y-intercept
    errors = np.sqrt(np.diag(covariance))
    
    calibrated_value = params[0]*voltage + params[1]
    error_calibrated_value = np.sqrt((errors[0]*voltage)**2 + (errors[1])**2)       # errors calculated using least-square method
    
    print(f'At the {voltage} V, the calibrated value with uncertainty for {axis} is {calibrated_value:.4f} ± {error_calibrated_value:.4f} µm.')
    return None

## Output

In [2]:
calibrate('x', 50)

At the 50 V, the calibrated value with uncertainty for x is 56.8627 ± 0.4649 µm.
